# Creates scatter plots for PIPS data relative to radar

In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import numpy.ma as ma
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.ticker as ticker
import matplotlib.dates as dates
import matplotlib.colors as colors
from mpl_toolkits.axes_grid1 import ImageGrid,make_axes_locatable,host_subplot
#from mpl_toolkits.basemap import Basemap
from datetime import datetime, timedelta
import sys
import os
import pyPIPS.utils as utils
import pyPIPS.thermolib as thermo
import pyPIPS.DSDlib as dsd
#import pyPIPS.disdrometer_module as dis
import pyPIPS.plotmodule as PIPSplot
#import pyPIPS.simulator as sim
import pyPIPS.pips_io as pipsio
import pyPIPS.PIPS as pips
import pyPIPS.parsivel_params as pp
import pyPIPS.parsivel_qc as pqc
import pyPIPS.polarimetric as dualpol
#from pyCRMtools.modules import plotmodule as plotmod
from pyCRMtools.modules import utils as CRMutils
from pyCRMtools.pycaps import arps_read
from pyCRMtools.pycaps import pycaps_fields
from pyCRMtools.pycaps import calvars_radar as radar
import pandas as pd
import xarray as xr
import glob
import numpy.random as random
from scipy.stats import gamma, uniform
from scipy.special import gamma as gammafunc
from scipy import ndimage
from metpy.plots import StationPlot
from metpy.calc import wind_components
from metpy.cbook import get_test_data
from metpy.plots import StationPlot
import matplotlib.colors as colors
from metpy.plots.wx_symbols import current_weather, sky_cover
from metpy.units import units
import matplotlib.colors as colors
import warnings
warnings.simplefilter('ignore')
%matplotlib inline

In [ ]:
from natsort import natsorted
from pprint import pprint

def remove_unneeded(ds):
    dim_names_to_remove = ['fields_KHTX', 'fields_KGWX', 'fields']
    ds = ds.drop_dims(dim_names_to_remove, errors='ignore')
    return ds
#

PIPS_dir = '/Volumes/scr_fast/Projects/VORTEXSE/obsdata/full_PIPS_dataset/'
# parsivel_filenames = glob.glob(PIPS_dir + '/parsivel_combined*60s*nc')
# parsivel_ds_read = xr.open_mfdataset(parsivel_filenames, combine='nested', concat_dim='time',
#                                      preprocess=pipsio.remove_unneeded) #, combine='nested', concat_dim='time')

parsivel_filenames = glob.glob(PIPS_dir + '/parsivel_combined_IOP4B_D1_2016*nc')
parsivel_ds_read_dict = {}

for parsivel_filename in parsivel_filenames:
    parsivel_ds_read = xr.open_dataset(parsivel_filename)
    parsivel_ds_read_dict[parsivel_ds_read.probe_name] = parsivel_ds_read

In [ ]:
# Filter entire dataset on rainrate > 0.1
parsivel_ds_dict = {}
for PIPS_name, parsivel_ds_read in parsivel_ds_read_dict.items():    
    parsivel_ds = parsivel_ds_read.where(parsivel_ds_read['rainrate_derived_qc'] > 0.1)
    # Filter on ZDR > 0.1 for now (Guifu's retrieval code uses their own emperical fits for ZDR < 0.1, so we need
    # to compute our own since we get weird artifacts using theirs)
    ND = parsivel_ds['ND_qc']
    dD = pp.parsivel_parameters['max_diameter_bins_mm'] - pp.parsivel_parameters['min_diameter_bins_mm']
    dualpol_dict = dualpol.calpolrain(10.7, '/Users/dawson29/Projects/pyPIPS/tmatrix/S-Band/SCTT_RAIN_fw100.dat',
                                  ND, dD)
    ZH = dualpol_dict['ZH']
    ZDR = dualpol_dict['ZDR']
    parsivel_ds['dBZ'] = dualpol_dict['REF']
    parsivel_ds['ZH'] = ZH
    parsivel_ds['ZDR'] = ZDR
    parsivel_ds = parsivel_ds.where(ZDR > 0.1)
    parsivel_ds_dict[PIPS_name] = parsivel_ds
    
# Drop PIPS1A because it is too far north
parsivel_list = [v for k, v in parsivel_ds_dict.items() if k not in 'PIPS1A']
parsivel_ds = xr.concat(parsivel_list, dim='time')
# Or... Just look at one PIPS
parsivel_ds = parsivel_ds_dict['PIPS1B']

In [ ]:
# Pick a PIPS for the cells below
# PIPS_name = 'PIPS2A'
# parsivel_ds = parsivel_ds_dict[PIPS_name]
# print(parsivel_ds)
# print(parsivel_ds['fields_KGWX'])
print(parsivel_ds['KGWX_beam_height_at_PIPS'].max())
print(parsivel_ds)
parsivel_ds['reflectivity_derived_qc'].plot()


In [ ]:
ND = parsivel_ds['ND_qc']
dD = pp.parsivel_parameters['max_diameter_bins_mm'] - pp.parsivel_parameters['min_diameter_bins_mm']
dualpol_dict = dualpol.calpolrain(10.7, '/Users/dawson29/Projects/pyPIPS/tmatrix/S-Band/SCTT_RAIN_fw100.dat',
                                  ND, dD)
ZH = dualpol_dict['ZH']
ZDR = dualpol_dict['ZDR']
rho = parsivel_ds['rho']
W_obs, _ = dsd.calc_lwc_qr_from_bins(ND, rho)
W_obs = W_obs * 1000.
W_retr_SATP = parsivel_ds['W_retr_SATP']
W_retr_TMM_F = parsivel_ds['W_retr_TMM_F']
W_retr_C08 = parsivel_ds['W_retr_C08']
W_retr_Z01 = parsivel_ds['W_retr_Z01']
Dm_obs = parsivel_ds['Dm43'] * 1000.
D0_obs = dsd.calc_D0_bin(ND.load()) * 1000.
Dm_retr_SATP = parsivel_ds['Dm_retr_SATP']
Dm_retr_TMM_F = parsivel_ds['Dm_retr_TMM_F']
Dm_retr_C08 = parsivel_ds['Dm_retr_C08']
Dm_retr_Z01 = parsivel_ds['Dm_retr_Z01']
RR_obs = parsivel_ds['rainrate_derived_qc']
Nt_obs = dsd.calc_NT_from_bins(parsivel_ds['ND_qc'])
Nt_retr_SATP = parsivel_ds['Nt_retr_SATP']
Nt_retr_C08 = parsivel_ds['Nt_retr_C08']
Nt_retr_Z01 = parsivel_ds['Nt_retr_Z01']
Nt_retr_TMM_F = parsivel_ds['Nt_retr_TMM_F']
RR_retr_SATP = parsivel_ds['RR_retr_SATP']
RR_retr_C08 = parsivel_ds['RR_retr_C08']
RR_retr_Z01 = parsivel_ds['RR_retr_Z01']
RR_retr_TMM_F = parsivel_ds['RR_retr_TMM_F']

# Radar retrieval parameters
rad_retr_dict = {}
for field in parsivel_ds['fields_KGWX'].values:
    print(field)
    rad_retr_dict[field] = parsivel_ds['KGWX_at_PIPS'].sel(fields_KGWX=field)
#W_retr_SATP_rad = parsivel_ds['KGWX_at_PIPS'].sel(fields_KGWX=)

In [ ]:
# Plot Z ono-to-one scatterplots
dBZ_rad = rad_retr_dict['REF_filtered']
dBZ_PIPS = parsivel_ds['dBZ']
print(dBZ_PIPS)

# The following is unnecessary
# goodindices = np.isfinite(dBZ_rad) & np.isfinite(dBZ_PIPS)
# print(goodindices)
# dBZ_rad = dBZ_rad.loc[goodindices]
# dBZ_PIPS = dBZ_PIPS.loc[goodindices]


dBZ_ds = xr.Dataset({'dBZ_rad': dBZ_rad, 'dBZ_PIPS': dBZ_PIPS})
axparams = {
    'var_lims': [[0.0, 60.], [0.0, 60.]],
    'alpha': 0.75,
    'markersize': 5,
    'label_x': r'$Z_H$ (PIPS; dBZ)',
    'label_y': r'$Z_H$ (KGWX; dBZ)',
}
fig, ax = PIPSplot.plot_one2one(dBZ_ds, 'dBZ_PIPS', 'dBZ_rad', axparams)
ax.set_title(title)

In [ ]:
# Plot ZDR ono-to-one scatterplots
ZDR_rad = rad_retr_dict['ZDR_filtered']
ZDR_PIPS = parsivel_ds['ZDR']

# The following is unnecessary
# goodindices = np.isfinite(dBZ_rad) & np.isfinite(dBZ_PIPS)
# print(goodindices)
# dBZ_rad = dBZ_rad.loc[goodindices]
# dBZ_PIPS = dBZ_PIPS.loc[goodindices]


ZDR_ds = xr.Dataset({'ZDR_rad': ZDR_rad, 'ZDR_PIPS': ZDR_PIPS})
axparams = {
    'var_lims': [[0.0, 3.], [0.0, 3.]],
    'alpha': 0.75,
    'markersize': 5,
    'label_x': r'$Z_{DR}$ (PIPS; dB)',
    'label_y': r'$Z_{DR}$ (KGWX; dB)',
}
fig, ax = PIPSplot.plot_one2one(ZDR_ds, 'ZDR_PIPS', 'ZDR_rad', axparams)
ax.set_title(title)

In [ ]:
# Plot Dm one-to-one scatterplots
Dm_retr_SATP = rad_retr_dict['Dm_SATP']
Dm_retr_C08 = rad_retr_dict['Dm_C08']
Dm_retr_Z01 = rad_retr_dict['Dm_Z01']
Dm_retr_TMM_F = rad_retr_dict['Dm_TMM_F']
for title, Dm_retr in zip(['VSE relation', 'C08 relation', 'Z01 relation', 'TMM (filtered) relation'], 
                          [Dm_retr_SATP, Dm_retr_C08, Dm_retr_Z01, Dm_retr_TMM_F]):
    overlap = (Dm_obs > 0.) & (Dm_retr > 0.)
    Dm_obs_plt = Dm_obs.where(overlap)
    Dm_retr_plt = Dm_retr.where(overlap)
    Dm_ds = xr.Dataset({'Dm_obs': Dm_obs_plt, 'Dm_retr': Dm_retr_plt, 'RR_obs': RR_obs})
    axparams = {
        'var_lims': [[0.0, 4.5], [0.0, 4.5]],
        'col_field': 'RR_obs',
        'col_field_lims': [0.1, 200.],
        'norm': colors.LogNorm(vmin=0.1, vmax=200.),
        'alpha': 0.75,
        'markersize': 5,
        'label_x': r'$D_m$ (obs; mm)',
        'label_y': r'$D_m$ (retrieved; mm)',
        'label_cb': r'$RR$ (mm h$^{-1}$)'
    }
    fig, ax = PIPSplot.plot_one2one(Dm_ds, 'Dm_obs', 'Dm_retr', axparams)
    ax.set_title(title)

In [ ]:
# Plot Nt one-to-one scatterplots
var_obs = Nt_obs
Nt_retr_SATP = rad_retr_dict['Nt_SATP']
Nt_retr_C08 = rad_retr_dict['Nt_C08']
Nt_retr_Z01 = rad_retr_dict['Nt_Z01']
Nt_retr_TMM_F = rad_retr_dict['Nt_TMM_F']
for title, var_retr in zip(['VSE relation', 'C08 relation', 'Z01 relation', 'TMM (filtered) relation'], 
                           [Nt_retr_SATP, Nt_retr_C08, Nt_retr_Z01, Nt_retr_TMM_F]):
    overlap = (var_obs > 0.) & (var_retr > 0.)
    var_obs_plt = var_obs.where(overlap)
    var_retr_plt = var_retr.where(overlap)
    var_ds = xr.Dataset({'var_obs': var_obs_plt, 'var_retr': var_retr_plt, 'RR_obs': RR_obs})
    axparams = {
        'var_lims': [[0.0, 4.5], [0.0, 4.5]],
        'plot_log': [True, True],
        'col_field': 'RR_obs',
        'col_field_lims': [0.1, 200.],
        'norm': colors.LogNorm(vmin=0.1, vmax=200.),
        'alpha': 0.75,
        'markersize': 5,
        'label_x': r'$N_T$ (obs; #/m3)',
        'label_y': r'$N_T$ (retrieved; #/m3)',
        'label_cb': r'$RR$ (mm h$^{-1}$)'
    }
    fig, ax = PIPSplot.plot_one2one(var_ds, 'var_obs', 'var_retr', axparams)
    ax.set_title(title)

In [ ]:
# Plot RR one-to-one scatterplots
var_obs = RR_obs
RR_retr_SATP = rad_retr_dict['RR_SATP']
RR_retr_C08 = rad_retr_dict['RR_C08']
RR_retr_Z01 = rad_retr_dict['RR_Z01']
RR_retr_TMM_F = rad_retr_dict['RR_TMM_F']
for title, var_retr in zip(['VSE relation', 'C08 relation', 'Z01 relation', 'TMM (filtered) relation'], 
                           [RR_retr_SATP, RR_retr_C08, RR_retr_Z01, RR_retr_TMM_F]):
    overlap = (var_obs > 0.) & (var_retr > 0.)
    var_obs_plt = var_obs.where(overlap)
    var_retr_plt = var_retr.where(overlap)
    var_ds = xr.Dataset({'var_obs': var_obs_plt, 'var_retr': var_retr_plt, 'RR_obs': RR_obs})
    axparams = {
        'var_lims': [[0.0, 100.], [0.0, 100.]],
        'plot_log': [False, False],
        'col_field': 'RR_obs',
        'col_field_lims': [0.1, 200.],
        'norm': colors.LogNorm(vmin=0.1, vmax=200.),
        'alpha': 0.75,
        'markersize': 5,
        'label_x': r'$RR$ (obs; mm h$^{-1}$)',
        'label_y': r'$RR$ (retrieved; mm h$^{-1}$)',
        'label_cb': r'$RR$ (mm h$^{-1}$)'
    }
    fig, ax = PIPSplot.plot_one2one(var_ds, 'var_obs', 'var_retr', axparams)
    ax.set_title(title)

In [ ]:
# Plot D0 vs ZDR and compute polynomial fit
ZDR = rad_retr_dict['ZDR_filtered']

overlap = (D0_obs > 0.) & (ZDR > 0.)
D0_plt = D0_obs.where(overlap)
ZDR_plt = ZDR.where(overlap)
RR_plt = RR_obs.where(overlap)

D0_ds = xr.Dataset({'D0': D0_plt, 'ZDR': ZDR_plt, 'RR_obs': RR_plt})

D0_plt_vals = D0_plt.values
ZDR_plt_vals = ZDR_plt.values

goodindices = np.isfinite(D0_plt_vals) & np.isfinite(ZDR_plt_vals)
D0_plt_vals = D0_plt_vals[goodindices]
ZDR_plt_vals = ZDR_plt_vals[goodindices]

In [ ]:

axparams = {
    'var_lims': [[0., 4.], [0., 5.]],
    'plot_log': [False, False],
    'col_field': 'RR_obs',
    'col_field_lims': [0.1, 200.],
    'norm': colors.LogNorm(vmin=0.1, vmax=200.),
    'alpha': 0.75,
    'markersize': 5,
    'label_x': r'$Z_{DR}$ (dB)',
    'label_y': r'$D_0 (mm)$', 
    'label_cb': r'$RR$ (mm h$^{-1}$)'
}
fig, ax = PIPSplot.plot_scatter(D0_ds, 'ZDR', 'D0', axparams)
poly_coeff, poly = dsd.calc_empirical_polyfit(ZDR_plt_vals, D0_plt_vals, order=3)
print(poly_coeff)

xx = np.linspace(0., 4.)
yy = poly(xx)
ax.plot(xx, yy, 'k')

D0_cao = 0.0436 * (xx**3.) - 0.216 * (xx**2.) + 1.076 * xx + 0.659
ax.plot(xx, D0_cao, 'b')

In [ ]:
# Plot log(R/Zh) one-to-one plot
overlap = (RR_obs > 0.) & (RR_retr_SATP > 0.)
RR_obs_plt = RR_obs.where(overlap)
RR_retr_SATP_plt = RR_retr_SATP.where(overlap)

ZH_plt = ZH.where(ZH > 0.)
RRdZH_obs = RR_obs_plt/ZH_plt
RRdZH_retr_SATP = RR_retr_SATP_plt/ZH_plt

RRdZH_ds = xr.Dataset({'RRdZH_obs': RRdZH_obs, 'RRdZH_retr_SATP': RRdZH_retr_SATP, 'RR_obs': RR_obs_plt})

axparams = {
    'var_lims': [[-4., -1.5], [-4., -1.5]],
    'plot_log': [True, True],
    'col_field': 'RR_obs',
    'col_field_lims': [0.1, 200.],
    'norm': colors.LogNorm(vmin=0.1, vmax=200.),
    'alpha': 0.75,
    'markersize': 5,
    'label_x': r'$\log{R/Z_H}$ (obs)',
    'label_y': r'$\log{R/Z_H}$ (retrieved)',
    'label_cb': r'$RR$ (mm h$^{-1}$)'
}
PIPSplot.plot_one2one(RRdZH_ds, 'RRdZH_obs', 'RRdZH_retr_SATP', axparams)

In [ ]:
# Plot R/Zh vs. ZDR and compute polynomial fit
overlap = (RRdZH_obs > 0.) & (ZDR > 0.)
RRdZH_obs_plt = RRdZH_obs.where(overlap)
ZDR_plt = ZDR.where(overlap)
RR_obs_plt = RR_obs.where(overlap)

RRdZH_ZDR_ds = xr.Dataset({'RRdZH': RRdZH_obs_plt, 'ZDR': ZDR_plt, 'RR_obs': RR_obs_plt})

RRdZH_vals = RRdZH_obs_plt.values
log_RRdZH_vals = np.log10(RRdZH_vals)
ZDR_vals = ZDR_plt.values

goodindices = np.isfinite(log_RRdZH_vals) & np.isfinite(ZDR_vals)
log_RRdZH_vals = log_RRdZH_vals[goodindices]
ZDR_vals = ZDR_vals[goodindices]

In [ ]:
from scipy.optimize import curve_fit
def func(x, a, b, c, d):
    return a*x**3. + b*x**2. + c*x + d

popt, pcov = curve_fit(func, ZDR_vals, log_RRdZH_vals)
print(popt)

# Gives almost the same results (reversed coefficient order...)

In [ ]:

axparams = {
    'var_lims': [[0., 4.], [-4., -1.5]],
    'plot_log': [False, True],
    'col_field': 'RR_obs',
    'col_field_lims': [0.1, 200.],
    'norm': colors.LogNorm(vmin=0.1, vmax=200.),
    'alpha': 0.75,
    'markersize': 5,
    'label_x': r'$Z_{DR}$ (dB)',
    'label_y': r'$\log{R/Z_H}$', 
    'label_cb': r'$RR$ (mm h$^{-1}$)'
}
fig, ax = PIPSplot.plot_scatter(RRdZH_ZDR_ds, 'ZDR', 'RRdZH', axparams)
poly_coeff, poly = dsd.calc_empirical_polyfit(ZDR_vals, log_RRdZH_vals, order=3)
print(poly_coeff)

xx = np.linspace(0., 4.)
yy = poly(xx)
ax.plot(xx, 10.**yy, 'k')

RdZH_cao = 10.**(-0.0363 * (xx**3.) + 0.316 * (xx**2.) - 1.178 * xx - 1.964)
ax.plot(xx, RdZH_cao, 'b')

In [ ]:
# Plot W/ZH one-to-one
overlap = (W_obs > 0.) & (W_retr_SATP > 0.)
W_obs_plt = W_obs.where(overlap)
W_retr_SATP_plt = W_retr_SATP.where(overlap)
RR_obs_plt = RR_obs.where(overlap)

WdZH_obs = W_obs_plt/ZH_plt
WdZH_retr_SATP = W_retr_SATP_plt/ZH_plt
WdZH_ds = xr.Dataset({'WdZH_obs': WdZH_obs, 'WdZH_retr': WdZH_retr_SATP, 'RR_obs': RR_obs_plt})

axparams = {
    'var_lims': [[-6., -2.], [-6, -2.]],
    'plot_log': [True, True],
    'col_field': 'RR_obs',
    'col_field_lims': [0.1, 200.],
    'norm': colors.LogNorm(vmin=0.1, vmax=200.),
    'alpha': 0.75,
    'markersize': 5,
    'label_x': r'$\log{W/Z_H}$ (obs)',
    'label_y': r'$\log{W/Z_H}$ (retrieved)',
    'label_cb': r'$RR$ (mm h$^{-1}$)'
}
PIPSplot.plot_one2one(WdZH_ds, 'WdZH_obs', 'WdZH_retr', axparams)

In [ ]:
# Compute empirical fit for W/ZH vs ZDR and plot

overlap = (W_obs > 0.)
W_obs_plt = W_obs.where(overlap)
WdZH_obs = W_obs_plt/ZH_plt
overlap = (WdZH_obs > 0.) & (ZDR > 0.)
WdZH_obs = WdZH_obs.where(overlap)
ZDR_plt = ZDR.where(overlap)
RR_obs_plt = RR_obs.where(overlap)

WdZH_ZDR_ds = xr.Dataset({'WdZH': WdZH_obs, 'ZDR': ZDR_plt, 'RR_obs': RR_obs_plt})

WdZH_vals = WdZH_obs.values
log_WdZH_vals = np.log10(WdZH_vals)
ZDR_vals = ZDR_plt.values

goodindices = np.isfinite(log_WdZH_vals) & np.isfinite(ZDR_vals)
log_WdZH_vals = log_WdZH_vals[goodindices]
ZDR_vals = ZDR_vals[goodindices]

In [ ]:

axparams = {
    'var_lims': [[0., 4.], [-6., -2.]],
    'plot_log': [False, True],
    'col_field': 'RR_obs',
    'col_field_lims': [0.1, 200.],
    'norm': colors.LogNorm(vmin=0.1, vmax=200.),
    'alpha': 0.75,
    'markersize': 5,
    'label_x': r'$Z_{DR}$ (dB)',
    'label_y': r'$\log{W/Z_H}$', 
    'label_cb': r'$RR$ (mm h$^{-1}$)'
}
fig, ax = PIPSplot.plot_scatter(WdZH_ZDR_ds, 'ZDR', 'WdZH', axparams)
poly_coeff, poly = dsd.calc_empirical_polyfit(ZDR_vals, log_WdZH_vals, order=3)
print(poly_coeff)

xx = np.linspace(0., 4.)
yy = poly(xx)
ax.plot(xx, 10.**yy, 'k')
WdZH_cao = 10.**(-0.0493 * (xx**3.) + 0.430 * (xx**2.) - 1.542 * xx - 3.019)
ax.plot(xx, WdZH_cao, 'b')

In [ ]:
# Plot Nt/ZH one-to-one
overlap = (Nt_obs > 0.) & (Nt_retr_SATP > 0.)
Nt_obs_plt = Nt_obs.where(overlap)
Nt_retr_SATP_plt = Nt_retr_SATP.where(overlap)
NtdZH_obs = Nt_obs_plt/ZH_plt
NtdZH_retr_SATP = Nt_retr_SATP_plt/ZH_plt
RR_obs_plt = RR_obs.where(overlap)

NtdZH_ds = xr.Dataset({'NtdZH_obs': NtdZH_obs, 'NtdZH_retr': NtdZH_retr_SATP, 'RR_obs': RR_obs_plt})

axparams = {
    'var_lims': [[-3., 2.], [-3., 2.]],
    'plot_log': [True, True],
    'col_field': 'RR_obs',
    'col_field_lims': [0.1, 200.],
    'norm': colors.LogNorm(vmin=0.1, vmax=200.),
    'alpha': 0.75,
    'markersize': 5,
    'label_x': r'$\log{N_T/Z_H}$ (obs)',
    'label_y': r'$\log{N_T/Z_H}$ (retrieved)',
    'label_cb': r'$RR$ (mm h$^{-1}$)'
}
PIPSplot.plot_one2one(NtdZH_ds, 'NtdZH_obs', 'NtdZH_retr', axparams)

In [ ]:
# Plot Z vs. ZDR
dBZ = dualpol_dict['REF']
dBZ_vals = dBZ.values
ZDR_vals = ZDR.values

goodindices = np.isfinite(dBZ_vals) & np.isfinite(ZDR_vals)
dBZ_vals = dBZ_vals[goodindices]
ZDR_vals = ZDR_vals[goodindices]

ZH_ZDR_ds = xr.Dataset({'ZH': dBZ, 'ZDR': ZDR, 'RR_obs': RR_obs})

axparams = {
    'var_lims': [[0., 60.], [0., 6.]],
    'plot_log': [False, False],
    'col_field': 'RR_obs',
    'col_field_lims': [0.1, 200.],
    'norm': colors.LogNorm(vmin=0.1, vmax=200.),
    'alpha': 0.75,
    'markersize': 5,
    'label_x': r'$Z_{H}$ (dBZ)',
    'label_y': r'$Z_{DR}$ (dB)', 
    'label_cb': r'$RR$ (mm h$^{-1}$)'
}
fig, ax = PIPSplot.plot_scatter(ZH_ZDR_ds, 'ZH', 'ZDR', axparams)
ax.set_aspect('auto')
poly_coeff, poly = dsd.calc_empirical_polyfit(dBZ_vals, ZDR_vals, order=2)
print(poly_coeff)

# xx = np.linspace(0., 4.)
# yy = poly(xx)
# ax.plot(xx, 10.**yy, 'k')
# WdZH_cao = 10.**(-0.0493 * (xx**3.) + 0.430 * (xx**2.) - 1.542 * xx - 3.019)
# ax.plot(xx, WdZH_cao, 'b')